In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#test1

import sys
import os
import pandas as pd
import numpy as np
import gc

mod = sys.modules[__name__]


# data load
path = 'drive/My Drive/bigcontest2019/data/'

test1_activity = pd.read_csv(path + 'test/test1_activity.csv').sort_values(['acc_id','day'])
# test1_payment = pd.read_csv(path + 'test/test1_payment.csv').sort_values(['acc_id','day'])
test1_trade = pd.read_csv(path + 'test/test1_trade.csv')
# test1_pledge = pd.read_csv(path + 'test/test1_pledge.csv').sort_values(['acc_id','day'])
# test1_combat = pd.read_csv(path + 'test/test1_combat.csv').sort_values(['acc_id','day'])

# function define
def merge_by_char_id(df_list):
    '''
    df_list
    '''
    df = eval(df_list[0])

    for arg in df_list[1:]:
        df_arg = eval(arg)
        df = pd.merge(df, df_arg,left_index=True, right_index=True, how='left')

    return df

def pivoting_df(df):
    ind = list(df.columns)[0]
    col = list(df.columns)[1]
    
    names = list(df.columns)[2:]

    for name in names:
        tmp = df.pivot(index = ind, columns= col,values=name).fillna(0)
        tmp.columns = [name+'_day'+str(x) for x in tmp.columns]

        setattr(mod, '{}'.format(name), tmp)

    final_df = merge_by_char_id(names)
    
    return final_df

def merge_final_df(df,*args):
    '''
    df : 기준이 되는 데이터 프레임 (char_id가 전부 있는 쪽으로!!)
    *args : 가변인자 (데이터 프레임을 원하는 만큼 집어넣으면 된다.)
    '''    
    df['char_id'] = list(df.index)
    df.reset_index(drop=True,inplace=True)
    
    for arg in args:
        arg['char_id'] = list(arg.index)
        arg.reset_index(drop=True,inplace=True)
        df = pd.merge(df, arg, how='left',on= 'char_id').fillna(0)
    
    return df


# activity
test1_activity['game_money_change'] = np.abs(test1_activity['game_money_change'])
test1_activity_group = test1_activity.groupby(['char_id','day']).agg({
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'}).reset_index(drop=False)
test1_activity_group.columns = ['activity_'+'_'.join(x) for x in test1_activity_group.columns.ravel()]
test1_activity_char = pivoting_df(test1_activity_group)

# # create char_id for payment
# id_label = test1_activity.groupby(['acc_id', 'char_id']).agg({'day': 'count'})
# id = test1_activity.groupby(['acc_id']).agg({'day': 'count'})
# label = id_label.div(id, level='acc_id') * 100
# label.reset_index(drop=False,inplace=True)
# label.rename(columns = {'day':'weight'},inplace=True)

# test1_weight = pd.merge(test1_payment,label,how='left',on='acc_id')
# test1_weight['amount_spent'] *= (test1_weight['weight']/100)
# test1_weight.drop(['acc_id','weight'],axis=1,inplace=True)

# # payment
# test1_payment_group = test1_weight.groupby(['char_id','day']).agg({'amount_spent':'sum'}).reset_index(drop=False)
# test1_payment_group.columns = ['payment_'+i for i in test1_payment_group.columns]

# test1_payment_char = pivoting_df(test1_payment_group)

# trade
## 판매자 테이블 정의
test1_trade_seller = test1_trade.groupby(['source_char_id','day']).agg({
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'target_char_id':'nunique',
                                         'item_type':'nunique',
                                         'item_amount':'sum',
                                         'item_price':'sum'}).reset_index(drop=False)

test1_trade_seller.columns = ['trade_seller_'+'_'.join(x) for x in test1_trade_seller.columns.ravel()]
test1_trade_seller['trade_seller_type_count'] -= test1_trade_seller['trade_seller_type_sum']
test1_trade_seller = test1_trade_seller.rename(columns = {'trade_seller_type_count':'trade_seller_type_personal','trade_seller_type_sum':'trade_seller_type_exchange'})

test1_trade_seller_char = pivoting_df(test1_trade_seller)

## 구매자 테이블 정의
test1_trade_buyer = test1_trade.groupby(['target_char_id','day']).agg({ 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                          'item_price':'sum'}).reset_index(drop=False)

test1_trade_buyer.columns = ['trade_buyer_'+'_'.join(x) for x in test1_trade_buyer.columns.ravel()]
test1_trade_buyer['trade_buyer_type_count'] -= test1_trade_buyer['trade_buyer_type_sum']
test1_trade_buyer = test1_trade_buyer.rename(columns = {'trade_buyer_type_count':'trade_buyer_type_personal','trade_buyer_type_sum':'trade_buyer_type_exchange'})

test1_trade_buyer_char= pivoting_df(test1_trade_buyer)

# # pledge
# test1_pledge_group = test1_pledge.groupby(['char_id','day']).agg(
#     {
#         'server' : 'nunique',               # 캐릭터 서버
#         'pledge_id' : 'nunique',            # 혈맹 아이디
#         'play_char_cnt' : 'sum',        # 게임에 접속한 혈맹원 수
#         'combat_char_cnt' : 'sum',      # 전투에 참여한 혈맹원 수
#         'pledge_combat_cnt': 'sum',     # 혈맹 간 전투 횟수의 합
#         'random_attacker_cnt' : 'sum',  # 혈맹원 중 막피 전투를 행한 횟수의 합
#         'random_defender_cnt': 'sum',   # 혈맹원 중 막피로부터 피해를 받은 횟수의 합
#         'same_pledge_cnt': 'sum',       # 동일 혈맹원 간 전투 횟수의 합
#         'temp_cnt' : 'sum',             # 혈맹원들의 단발성 전투 횟수의 합
#         'etc_cnt' : 'sum',              # 혈맹원들의 기타 전투 횟수의 합
#         'combat_play_time': 'sum',      # 혈맹의 전투 캐릭터들의 플레이 시간의 합
#         'non_combat_play_time' : 'sum' # 혈맹의 非전투 캐릭터 플레이 시간의 합
#     }).reset_index(drop=False)

# test1_pledge_group.columns = ['pledge_'+x for x in test1_pledge_group.columns]
# test1_pledge_char = pivoting_df(test1_pledge_group)

# # combat
# test1_combat_group = test1_combat.groupby(['char_id','day']).agg(
#     {
#         'server' : 'nunique',          # 캐릭터 서버
#         'class' : 'nunique',           # 직업
#         'pledge_cnt' : 'sum',          # 혈맹간 전투에 참여한 횟수
#         'random_attacker_cnt' : 'sum', # 본인이 막피 공격을 행한 횟수
#         'random_defender_cnt' : 'sum', # 막피 공격자로부터 공격을 받은 횟수
#         'temp_cnt' : 'sum',            # 단발성 전투 횟수
#         'same_pledge_cnt' : 'sum',     # 동일 혈맹원 간의 전투 횟수
#         'etc_cnt' : 'sum',             # 기타 전투 횟수
#         'num_opponent' : 'sum'         # 전투 상대 캐릭터수
#     }).reset_index(drop=False)

# test1_combat_group.columns = ['combat_'+x for x in test1_combat_group.columns]
# test1_combat_char = pivoting_df(test1_combat_group)

# Final Merge
test1_char = merge_final_df(test1_activity_char ,
                # test1_payment_char,
                test1_trade_seller_char,
                test1_trade_buyer_char )#,
                # test1_pledge_char,
                # test1_combat_char)

In [0]:
test1_char.shape

(74603, 897)

In [0]:
# os.chdir('drive/My Drive/bigcontest2019/scripts/model/BJW/Final/')

test1_char.to_csv('test1_char_small.csv',encoding='utf-8',index=False)

In [0]:
os.chdir('../')
os.getcwd()

'/content'

In [0]:
# test2
import sys
import os
import pandas as pd
import numpy as np
import gc

mod = sys.modules[__name__]


# data load
path = 'drive/My Drive/bigcontest2019/data/'
test2_activity = pd.read_csv(path + 'test/test2_activity.csv').sort_values(['acc_id','day'])
test2_payment = pd.read_csv(path + 'test/test2_payment.csv').sort_values(['acc_id','day'])
test2_trade = pd.read_csv(path + 'test/test2_trade.csv')
test2_pledge = pd.read_csv(path + 'test/test2_pledge.csv').sort_values(['acc_id','day'])
test2_combat = pd.read_csv(path + 'test/test2_combat.csv').sort_values(['acc_id','day'])
# function define
def merge_by_char_id(df_list):
    '''
    df_list
    '''
    df = eval(df_list[0])

    for arg in df_list[1:]:
        df_arg = eval(arg)
        df = pd.merge(df, df_arg,left_index=True, right_index=True, how='left')

    return df

def pivoting_df(df):
    ind = list(df.columns)[0]
    col = list(df.columns)[1]
    
    names = list(df.columns)[2:]

    for name in names:
        tmp = df.pivot(index = ind, columns= col,values=name).fillna(0)
        tmp.columns = [name+'_day'+str(x) for x in tmp.columns]

        setattr(mod, '{}'.format(name), tmp)

    final_df = merge_by_char_id(names)
    
    return final_df

def merge_final_df(df,*args):
    '''
    df : 기준이 되는 데이터 프레임 (char_id가 전부 있는 쪽으로!!)
    *args : 가변인자 (데이터 프레임을 원하는 만큼 집어넣으면 된다.)
    '''    
    df['char_id'] = list(df.index)
    df.reset_index(drop=True,inplace=True)
    
    for arg in args:
        arg['char_id'] = list(arg.index)
        arg.reset_index(drop=True,inplace=True)
        df = pd.merge(df, arg, how='left',on= 'char_id').fillna(0)
    
    return df


# activity
test2_activity['game_money_change'] = np.abs(test2_activity['game_money_change'])
test2_activity_group = test2_activity.groupby(['char_id','day']).agg({
                                                             'server':'nunique', 
                                                             'playtime':['sum','mean'], 
                                                             'npc_kill':['sum','mean'], 
                                                             'solo_exp':'sum',
                                                             'party_exp':'sum', 
                                                             'quest_exp':'sum',
                                                             'rich_monster':'sum', 
                                                             'death':'sum', 
                                                             'revive':'sum',
                                                             'exp_recovery':'sum',
                                                             'fishing':'sum',
                                                             'private_shop':'sum',
                                                             'game_money_change':'sum',
                                                             'enchant_count':'sum'}).reset_index(drop=False)
test2_activity_group.columns = ['activity_'+'_'.join(x) for x in test2_activity_group.columns.ravel()]
test2_activity_char = pivoting_df(test2_activity_group)

# create char_id for payment
id_label = test2_activity.groupby(['acc_id', 'char_id']).agg({'day': 'count'})
id = test2_activity.groupby(['acc_id']).agg({'day': 'count'})
label = id_label.div(id, level='acc_id') * 100
label.reset_index(drop=False,inplace=True)
label.rename(columns = {'day':'weight'},inplace=True)

test2_weight = pd.merge(test2_payment,label,how='left',on='acc_id')
test2_weight['amount_spent'] *= (test2_weight['weight']/100)
test2_weight.drop(['acc_id','weight'],axis=1,inplace=True)

# payment
test2_payment_group = test2_weight.groupby(['char_id','day']).agg({'amount_spent':'sum'}).reset_index(drop=False)
test2_payment_group.columns = ['payment_'+i for i in test2_payment_group.columns]

test2_payment_char = pivoting_df(test2_payment_group)

# trade
## 판매자 테이블 정의
test2_trade_seller = test2_trade.groupby(['source_char_id','day']).agg({
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'target_char_id':'nunique',
                                         'item_type':'nunique',
                                         'item_amount':'sum',
                                         'item_price':'sum'}).reset_index(drop=False)

test2_trade_seller.columns = ['trade_seller_'+'_'.join(x) for x in test2_trade_seller.columns.ravel()]
test2_trade_seller['trade_seller_type_count'] -= test2_trade_seller['trade_seller_type_sum']
test2_trade_seller = test2_trade_seller.rename(columns = {'trade_seller_type_count':'trade_seller_type_personal','trade_seller_type_sum':'trade_seller_type_exchange'})

test2_trade_seller_char = pivoting_df(test2_trade_seller)

## 구매자 테이블 정의
test2_trade_buyer = test2_trade.groupby(['target_char_id','day']).agg({ 
                                         'type':['nunique','count','sum'], # 거래의 종류들을 파악하기 위해서 -> nuique = 2이면 두 종류의 거래 모두 진행 / count 
                                         'server':'nunique', 
                                         'source_char_id':'nunique', # 몇개의 캐릭터 운용하는지
                                         'item_type':'nunique', 
                                         'item_amount':'sum',
                                          'item_price':'sum'}).reset_index(drop=False)

test2_trade_buyer.columns = ['trade_buyer_'+'_'.join(x) for x in test2_trade_buyer.columns.ravel()]
test2_trade_buyer['trade_buyer_type_count'] -= test2_trade_buyer['trade_buyer_type_sum']
test2_trade_buyer = test2_trade_buyer.rename(columns = {'trade_buyer_type_count':'trade_buyer_type_personal','trade_buyer_type_sum':'trade_buyer_type_exchange'})

test2_trade_buyer_char= pivoting_df(test2_trade_buyer)

# pledge
test2_pledge_group = test2_pledge.groupby(['char_id','day']).agg(
    {
        'server' : 'nunique',               # 캐릭터 서버
        'pledge_id' : 'nunique',            # 혈맹 아이디
        'play_char_cnt' : 'sum',        # 게임에 접속한 혈맹원 수
        'combat_char_cnt' : 'sum',      # 전투에 참여한 혈맹원 수
        'pledge_combat_cnt': 'sum',     # 혈맹 간 전투 횟수의 합
        'random_attacker_cnt' : 'sum',  # 혈맹원 중 막피 전투를 행한 횟수의 합
        'random_defender_cnt': 'sum',   # 혈맹원 중 막피로부터 피해를 받은 횟수의 합
        'same_pledge_cnt': 'sum',       # 동일 혈맹원 간 전투 횟수의 합
        'temp_cnt' : 'sum',             # 혈맹원들의 단발성 전투 횟수의 합
        'etc_cnt' : 'sum',              # 혈맹원들의 기타 전투 횟수의 합
        'combat_play_time': 'sum',      # 혈맹의 전투 캐릭터들의 플레이 시간의 합
        'non_combat_play_time' : 'sum' # 혈맹의 非전투 캐릭터 플레이 시간의 합
    }).reset_index(drop=False)

test2_pledge_group.columns = ['pledge_'+x for x in test2_pledge_group.columns]
test2_pledge_char = pivoting_df(test2_pledge_group)

# combat
test2_combat_group = test2_combat.groupby(['char_id','day']).agg(
    {
        'server' : 'nunique',          # 캐릭터 서버
        'class' : 'nunique',           # 직업
        'pledge_cnt' : 'sum',          # 혈맹간 전투에 참여한 횟수
        'random_attacker_cnt' : 'sum', # 본인이 막피 공격을 행한 횟수
        'random_defender_cnt' : 'sum', # 막피 공격자로부터 공격을 받은 횟수
        'temp_cnt' : 'sum',            # 단발성 전투 횟수
        'same_pledge_cnt' : 'sum',     # 동일 혈맹원 간의 전투 횟수
        'etc_cnt' : 'sum',             # 기타 전투 횟수
        'num_opponent' : 'sum'         # 전투 상대 캐릭터수
    }).reset_index(drop=False)

test2_combat_group.columns = ['combat_'+x for x in test2_combat_group.columns]
test2_combat_char = pivoting_df(test2_combat_group)

# Final Merge
test2_char = merge_final_df(test2_activity_char ,
                test2_payment_char,
                test2_trade_seller_char,
                test2_trade_buyer_char ,
                test2_pledge_char,
                test2_combat_char)



In [0]:
train_label = train_label = pd.read_csv(path + 'train/train_label.csv')
train_label
# label

,acc_id,survival_time,amount_spent
0,27835,64,0.002559
1,12351,64,0.120154
2,125437,55,0.182593
3,104483,64,0.016241
4,4704,20,0.226396
5,3277,50,0.058863
6,118874,64,0.030777
7,7678,5,0.000000
8,124997,46,0.009187
9,116489,23,0.000000


In [0]:
os.chdir('drive/My Drive/bigcontest2019/scripts/model/BJW/Final/')

test2_char.to_csv('test2_char.csv',encoding='utf-8',index=False)

In [0]:
oof1_time, oof1_spent, pred1_time, pred1_spent = lgb_model(
    train_merge, test1_merge, train_label.sort_values('acc_id').reset_index(drop=True), FOLDS, SEED)